In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

## 2024 데이터 전처리

In [2]:
df = pd.read_excel('data/2024/2024 외래관광객조사_Data.xlsx')

- 국적: D_NAT
- 체류기간: M일HAP
- 동반자 유무: Q7A
- 동행자 유형: Q7a_dk ~ Q7a8
- 여행 형태: D_GUB
- 방문 목적: D_MOK
- 총지출: 총액1인TOT항공제외2
- 참여한 활동(자연경관 감상, 뷰티/미용 관광): Q8a03, Q8a12
- 참여 만족도: Q8_1a1, Q8_1a2, Q8_1a3
- 가중치: weight 

In [ ]:
cols_needed = ['D_NAT','M일HAP', 
               'Q7A','Q7a_dk', 'Q7a2', 'Q7a3', 'Q7a4', 'Q7a5', 'Q7a6', 'Q7a7', 'Q7a8',
               'D_GUB', 'D_MOK', '총액1인TOT_개별국제교통비제외2', 
               'Q8a03', 'Q8a12', 
               'Q8_1a1', 'Q8_1a2', 'Q8_1a3']
df = df[cols_needed]

In [4]:
df.isnull().sum()

D_NAT                     0
M일HAP                     0
Q7A                       0
Q7a_dk                11275
Q7a2                  12905
Q7a3                  15081
Q7a4                  14707
Q7a5                  14518
Q7a6                  11961
Q7a7                  14520
Q7a8                  16108
D_GUB                     0
D_MOK                     0
총액1인TOT_개별국제교통비제외2        0
Q8a03                  6934
Q8a12                 13632
Q8_1a1                    0
Q8_1a2                 1025
Q8_1a3                 2947
dtype: int64

### 동행자 유형 분류

In [ ]:
# 1. 동행자 관련 컬럼만 추출
companion_cols = ['Q7a_dk', 'Q7a2', 'Q7a3', 'Q7a4', 'Q7a5', 'Q7a6', 'Q7a7', 'Q7a8']

# 2. 결측치 처리 안함 
# 결측치는 <선택하지 않음>을 의미하므로 처리하지 않음

# 3. 동행자유형 파생 변수 만들기
def classify_companion(row):
    # 혼자 온 경우
    if row['Q7a_dk'] == 1:
        return '혼자'

    # 가족 유형
    if any([
        row['Q7a2'] == 2,
        row['Q7a3'] == 3,
        row['Q7a4'] == 4,
        row['Q7a5'] == 5
    ]):
        return '가족'

    # 친구
    if row['Q7a6'] == 6:
        return '친구'

    # 직장동료
    if row['Q7a7'] == 7:
        return '직장동료'

    # 기타
    if row['Q7a8'] == 8:
        return '기타'

    # 아무 것도 선택하지 않은 경우
    return '응답없음'

# 4. 적용
df['동행자유형'] = df.apply(classify_companion, axis=1)

# 5. 유효 응답만 남기기
df = df[df['동행자유형'] != '응답없음'].copy()


In [6]:
df.columns

Index(['D_NAT', 'M일HAP', 'Q7A', 'Q7a_dk', 'Q7a2', 'Q7a3', 'Q7a4', 'Q7a5',
       'Q7a6', 'Q7a7', 'Q7a8', 'D_GUB', 'D_MOK', '총액1인TOT_개별국제교통비제외2', 'Q8a03',
       'Q8a12', 'Q8_1a1', 'Q8_1a2', 'Q8_1a3', '동행자유형'],
      dtype='object')

In [7]:
df['동행자유형'].value_counts()

동행자유형
가족      5775
혼자      4941
친구      3902
직장동료    1530
기타        68
Name: count, dtype: int64

### 국적 선별 
- 2023년 글로벌 방한 관광객 통계 기준 상위 6개국 
- GWI-Rankings 상위 5개국 중 2개국(방한관광객 상위 국가와 중복 제외)

In [8]:
df['D_NAT'].value_counts()

D_NAT
1     2215
2     2007
3     1300
4     1192
5      834
7      769
10     687
6      648
12     644
9      621
97     607
8      606
13     536
14     536
15     530
19     451
11     425
18     423
17     419
16     419
20     347
Name: count, dtype: int64

In [ ]:
df['D_NAT2'].value_counts()

### 국가명 매핑

In [9]:
df['D_NAT'].dtype

dtype('int64')

In [ ]:
nat_map = {
    1: '중국',
    2: '일본',
    3: '대만',
    4: '미국',
    5: '홍콩',
    6: '태국',
    7: '베트남',
    10: '싱가포르',
    16: '영국',
    18: '독일',
    12: '중동'
}

df['국가명'] = df['D_NAT'].map(nat_map)

# 분석 대상 국가만 필터링
df = df[df['국가명'].notna()]

In [11]:
df['국가명'].value_counts()

국가명
중국      2215
일본      2007
대만      1300
미국      1192
홍콩       834
베트남      769
싱가포르     687
태국       648
중동       644
독일       423
영국       419
Name: count, dtype: int64

### 체류일수 및 총지출 0 이하 제거

In [12]:
df = df[(df['M일HAP'] > 0) & (df['총액1인TOT_개별국제교통비제외2'] > 0)]

### 웰니스 활동 참여 여부 파생변수 생성

In [13]:
df['Q8a03'].dtype

dtype('float64')

In [14]:
df['Q8a12'].dtype

dtype('float64')

In [15]:
df['웰니스참여'] = ((df['Q8a03'] == 3) | (df['Q8a12'] == 12)).astype(int)

In [16]:
df['웰니스참여'].value_counts()

웰니스참여
1    6584
0    4554
Name: count, dtype: int64

In [17]:
df['Q8_1a1'].value_counts()

Q8_1a1
2     2127
1     2081
3     1483
5     1085
18     812
8      524
13     417
12     384
6      363
7      298
14     275
11     271
4      256
19     224
17     121
9      119
16     110
10      69
20      64
15      55
Name: count, dtype: int64

In [18]:
df['Q8_1a2'].value_counts()

Q8_1a2
1.0     3060
2.0     2674
3.0     1315
5.0      937
6.0      349
7.0      347
4.0      319
12.0     266
11.0     208
13.0     203
8.0      181
18.0     164
9.0       90
14.0      78
16.0      58
10.0      48
15.0      47
17.0      42
19.0      32
20.0      13
Name: count, dtype: int64

In [19]:
df['Q8_1a3'].value_counts()

Q8_1a3
2.0     2003
1.0     1898
3.0     1361
5.0      960
7.0      410
4.0      357
6.0      341
13.0     320
12.0     281
11.0     265
8.0      189
18.0     174
14.0     124
10.0      66
9.0       65
19.0      47
16.0      41
15.0      40
17.0      31
20.0      19
Name: count, dtype: int64

- 자연경관(3)은 Top3 만족 활동 중 매우 자주 등장 (총 4000회 이상)

- 뷰티/미용(12)도 상대적으로 빈도는 적지만 분명히 선택되고 있음

### 웰니스 활동이 Top3 만족 활동에 포함된 총 응답자 수
- 현재는 각 코드별 총 등장 수만 계산됨 -> 실제 몇 명이 ‘3 또는 12’를 한번 이상 선택했는지 계산

In [20]:
df['웰니스만족'] = df[['Q8_1a1', 'Q8_1a2', 'Q8_1a3']].apply(
    lambda row: any(x in [3, 12] for x in row if pd.notnull(x)),
    axis=1
).astype(int)

df['웰니스만족'].value_counts()

웰니스만족
0    6251
1    4887
Name: count, dtype: int64

### 여행 목적 컬럼 (D_MOK) 전처리
- 여가/위락/힐링, 친구/친지 방문, 사업/전문 활동, 교육, 기타
- 여가/힐링을 즐길 의사와 여건이 있었던 응답자만 포함

In [21]:
df['D_MOK'].dtype

dtype('int64')

In [22]:
df = df[df['D_MOK'].isin([1, 2])]

In [23]:
df['D_MOK'].value_counts()

D_MOK
1    7374
2    1395
Name: count, dtype: int64

In [24]:
# 분류 성공 여부 및 결측 확인
# 결측치 확인
print("동행자유형 결측치 수:", df['동행자유형'].isnull().sum())

# 고유값 확인
print("동행자유형 고유값 목록:", df['동행자유형'].dropna().unique())

동행자유형 결측치 수: 0
동행자유형 고유값 목록: ['가족' '혼자' '친구' '직장동료' '기타']


- ANOVA 검정 안정 확보를 위해 로그 변환 변수 추가

In [25]:
df['log_총액'] = np.log(df['총액1인TOT_개별국제교통비제외2'])

### 24년도 전처리 데이터 저장

In [26]:
df['year'] = 2024  # 연도 컬럼 붙이기
df.to_csv("data/2024/외래관광객조사_2024_전처리2.csv", index=False, encoding='utf-8-sig')